In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [148]:
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", None)

In [149]:
enrol_df = pd.read_csv(r"D:\UIDAI hackathon\JandK\enrollment1.csv")
enrol_df = pd.concat([enrol_df, pd.read_csv(r"D:\UIDAI hackathon\JandK\enrollment2.csv")], ignore_index=True)
enrol_df = pd.concat([enrol_df, pd.read_csv(r"D:\UIDAI hackathon\JandK\enrollment3.csv")], ignore_index=True)

demo_df = pd.read_csv(r"D:\UIDAI hackathon\JandK\demographic1.csv")
demo_df = pd.concat([demo_df, pd.read_csv(r"D:\UIDAI hackathon\JandK\demographic2.csv")], ignore_index=True)

bio_df = pd.read_csv(r"D:\UIDAI hackathon\JandK\biometric1.csv")
bio_df = pd.concat([bio_df, pd.read_csv(r"D:\UIDAI hackathon\JandK\biometric2.csv")], ignore_index=True)

In [150]:
all_dfs = [enrol_df, demo_df, bio_df]

In [151]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,02-09-2025,Jammu & Kashmir,Baramula,193101,3,0,0
1,04-09-2025,Jammu & Kashmir,Jammu,180002,0,1,0
2,09-09-2025,Jammu & Kashmir,Jammu,181102,1,0,0
3,12-09-2025,Jammu & Kashmir,Jammu,180010,1,0,0
4,19-09-2025,Jammu & Kashmir,Kupwara,193224,1,1,0


In [152]:
enrol_df["date"] = pd.to_datetime(enrol_df["date"], dayfirst=True)
demo_df["date"] = pd.to_datetime(demo_df["date"], dayfirst=True)
bio_df["date"] = pd.to_datetime(bio_df["date"], dayfirst=True)

In [153]:
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())

28 27 32


In [154]:
enrol_df["district"].value_counts().sort_index()

district
Anantnag         908
Badgam           444
Bandipore        311
Bandipur           2
Baramula         963
Budgam           348
Doda             732
Ganderbal        283
Jammu           2175
Kargil           288
Kathua           981
Kishtwar         227
Kulgam           185
Kupwara          521
Leh              124
Leh (ladakh)       2
Pulwama          563
Punch            333
Rajauri            3
Rajouri          800
Ramban           344
Reasi            441
Samba            335
Shopian            1
Shupiyan          81
Srinagar        1385
Udhampur         886
punch              1
Name: count, dtype: int64

In [155]:
bio_df["district"].value_counts().sort_index()

district
?                  2
Anantnag        3203
Badgam          1765
Bandipore       1110
Bandipur          11
Baramula        3523
Baramulla          3
Budgam          1554
Doda            2795
Ganderbal       1050
Jammu           9867
Kargil          1255
Kathua          3914
Kishtwar         678
Kulgam           863
Kupwara         1557
Leh              994
Leh (ladakh)      39
Poonch             8
Pulwama         2349
Punch            942
Rajauri           68
Rajouri         2831
Ramban          1118
Reasi           1703
Samba           1803
Shupiyan         250
Srinagar        5657
Udhampur        3829
\                  1
punch              1
udhampur           3
Name: count, dtype: int64

In [156]:
demo_df["district"].value_counts().sort_index()

district
Anantnag        1453
Badgam           794
Bandipore        461
Bandipur           2
Baramula        1604
Budgam           683
Doda            1199
Ganderbal        498
Jammu           4039
Kargil           598
Kathua          1654
Kishtwar         301
Kulgam           333
Kupwara          706
Leh              367
Leh (ladakh)      12
Poonch             2
Pulwama         1051
Punch            428
Rajauri           24
Rajouri         1245
Ramban           484
Reasi            702
Samba            725
Shupiyan         114
Srinagar        2512
Udhampur        1594
Name: count, dtype: int64

In [157]:
cleanup_map = {
    "Badgam": "Budgam",
    "Bandipur": "Bandipora",
    "Baramula": "Baramulla",
    "Leh (ladakh)": "Leh",
    "Punch": "Poonch",
    "punch": "Poonch",
    "Rajauri": "Rajouri",
    "Shupiyan": "Shopian",
    "udhampur": "Udhampur",
    "Jammu & Kashmir":"Jammu and Kashmir"
    
}

noise = ["?", "\\"]

for df in all_dfs:
    df["district"] = df["district"].replace(cleanup_map)
    df["state"] = df["state"].replace(cleanup_map)

enrol_df = enrol_df[~enrol_df["district"].isin(noise)]
demo_df = demo_df[~demo_df["district"].isin(noise)]
bio_df = bio_df[~bio_df["district"].isin(noise)]

all_dfs = [enrol_df, demo_df, bio_df]
    
print(enrol_df["district"].nunique(), demo_df["district"].nunique(), bio_df["district"].nunique())

23 23 23


In [158]:
enrol_df["district"] = enrol_df["district"].str.lower()
demo_df["district"] = demo_df["district"].str.lower()
bio_df["district"] = bio_df["district"].str.lower()

C:\Users\yasse\AppData\Local\Temp\ipykernel_1796\2095086038.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bio_df["district"] = bio_df["district"].str.lower()


In [159]:
print(enrol_df["pincode"].nunique(), demo_df["pincode"].nunique(), bio_df["pincode"].nunique())

202 206 207


In [160]:
# Analyze pincode differences
enrol_pincodes = set(enrol_df["pincode"].dropna())
demo_pincodes = set(demo_df["pincode"].dropna())
bio_pincodes = set(bio_df["pincode"].dropna())

print("Enrollment pincodes:", len(enrol_pincodes))
print("Demographic pincodes:", len(demo_pincodes))
print("Biometric pincodes:", len(bio_pincodes))
print()

# Find differences
only_in_enrol = enrol_pincodes - demo_pincodes - bio_pincodes
only_in_demo = demo_pincodes - enrol_pincodes - bio_pincodes
only_in_bio = bio_pincodes - enrol_pincodes - demo_pincodes
in_all = enrol_pincodes & demo_pincodes & bio_pincodes

print(f"Pincodes only in enrollment: {len(only_in_enrol)}")
print(f"Pincodes only in demographic: {len(only_in_demo)}")
print(f"Pincodes only in biometric: {len(only_in_bio)}")
print(f"Pincodes in all three: {len(in_all)}")
print()

# Check for NULL/NaN values
print("NULL pincodes:")
print(f"Enrollment: {enrol_df["pincode"].isna().sum()}")
print(f"Demographic: {demo_df["pincode"].isna().sum()}")
print(f"Biometric: {bio_df["pincode"].isna().sum()}")
print()

# Show some examples of unique pincodes
print("Sample pincodes only in enrollment:", list(only_in_enrol))
print("Sample pincodes only in demographic:", list(only_in_demo))
print("Sample pincodes only in biometric:", list(only_in_bio))

Enrollment pincodes: 202
Demographic pincodes: 206
Biometric pincodes: 207

Pincodes only in enrollment: 0
Pincodes only in demographic: 0
Pincodes only in biometric: 1
Pincodes in all three: 202

NULL pincodes:
Enrollment: 0
Demographic: 0
Biometric: 0

Sample pincodes only in enrollment: []
Sample pincodes only in demographic: []
Sample pincodes only in biometric: [181209]


In [161]:
for df in all_dfs:
    df["month"] = df["date"].dt.month

In [162]:
enrol_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13667 entries, 0 to 13666
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            13667 non-null  datetime64[ns]
 1   state           13667 non-null  object        
 2   district        13667 non-null  object        
 3   pincode         13667 non-null  int64         
 4   age_0_5         13667 non-null  int64         
 5   age_5_17        13667 non-null  int64         
 6   age_18_greater  13667 non-null  int64         
 7   month           13667 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(4), object(2)
memory usage: 800.9+ KB


In [163]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23585 entries, 0 to 23584
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           23585 non-null  datetime64[ns]
 1   state          23585 non-null  object        
 2   district       23585 non-null  object        
 3   pincode        23585 non-null  int64         
 4   demo_age_5_17  23585 non-null  int64         
 5   demo_age_17_   23585 non-null  int64         
 6   month          23585 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 1.2+ MB


In [164]:
bio_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54743 entries, 0 to 54745
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          54743 non-null  datetime64[ns]
 1   state         54743 non-null  object        
 2   district      54743 non-null  object        
 3   pincode       54743 non-null  int64         
 4   bio_age_5_17  54743 non-null  int64         
 5   bio_age_17_   54743 non-null  int64         
 6   month         54743 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(3), object(2)
memory usage: 3.1+ MB


In [165]:
enrol_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,month
0,2025-09-02,Jammu and Kashmir,baramulla,193101,3,0,0,9
1,2025-09-04,Jammu and Kashmir,jammu,180002,0,1,0,9
2,2025-09-09,Jammu and Kashmir,jammu,181102,1,0,0,9
3,2025-09-12,Jammu and Kashmir,jammu,180010,1,0,0,9
4,2025-09-19,Jammu and Kashmir,kupwara,193224,1,1,0,9


In [166]:
demo_df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,month
0,2025-09-05,Jammu and Kashmir,jammu,181205,1,0,9
1,2025-09-06,Jammu and Kashmir,baramulla,193101,0,1,9
2,2025-09-06,Jammu and Kashmir,kathua,184101,0,1,9
3,2025-09-08,Jammu and Kashmir,baramulla,193101,0,1,9
4,2025-09-09,Jammu and Kashmir,jammu,180001,0,2,9


In [167]:
bio_df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_,month
0,2025-06-26,Jammu and Kashmir,jammu,180002,0,1,6
1,2025-06-26,Jammu and Kashmir,kargil,194103,0,1,6
2,2025-06-27,Jammu and Kashmir,baramulla,193122,0,1,6
3,2025-06-21,Jammu and Kashmir,doda,182204,0,1,6
4,2025-06-28,Jammu and Kashmir,doda,182204,0,1,6


In [168]:
enrol_agg = enrol_df.groupby(["state", "district", "month"])[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
demo_agg = demo_df.groupby(["state", "district", "month"])[["demo_age_5_17", "demo_age_17_"]].sum().reset_index()
bio_agg = bio_df.groupby(["state", "district", "month"])[["bio_age_5_17", "bio_age_17_"]].sum().reset_index()

combined_df = enrol_agg.merge(demo_agg, on = ["state", "district", "month"], how = "left").merge(bio_agg, on = ["state", "district", "month"], how = "left")
combined_df.fillna(0, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0
3,Jammu and Kashmir,anantnag,6,419,70,2,537.0,3257.0,3462.0,3897.0
4,Jammu and Kashmir,anantnag,9,861,104,1,455.0,2741.0,3951.0,2256.0


In [169]:
combined_df["E"] = combined_df["age_0_5"] + combined_df["age_5_17"] + combined_df["age_18_greater"]
combined_df["DU"] = combined_df["demo_age_5_17"] + combined_df["demo_age_17_"]
combined_df["BU"] = combined_df["bio_age_5_17"] + combined_df["bio_age_17_"]
combined_df["U"] = combined_df["DU"] + combined_df["BU"]
combined_df["T"] = combined_df["E"] + combined_df["U"]
combined_df.head(20)

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0
3,Jammu and Kashmir,anantnag,6,419,70,2,537.0,3257.0,3462.0,3897.0,491,3794.0,7359.0,11153.0,11644.0
4,Jammu and Kashmir,anantnag,9,861,104,1,455.0,2741.0,3951.0,2256.0,966,3196.0,6207.0,9403.0,10369.0
5,Jammu and Kashmir,anantnag,10,670,87,3,353.0,2311.0,5244.0,2980.0,760,2664.0,8224.0,10888.0,11648.0
6,Jammu and Kashmir,anantnag,11,744,93,24,908.0,5329.0,3388.0,2037.0,861,6237.0,5425.0,11662.0,12523.0
7,Jammu and Kashmir,anantnag,12,655,56,6,1155.0,6486.0,13494.0,4249.0,717,7641.0,17743.0,25384.0,26101.0
8,Jammu and Kashmir,bandipora,10,1,0,0,0.0,1.0,2.0,3.0,1,1.0,5.0,6.0,7.0
9,Jammu and Kashmir,bandipora,12,1,0,0,0.0,1.0,0.0,0.0,1,1.0,0.0,1.0,2.0


In [170]:
district_monthly_counts = combined_df.groupby(["district", "month"]).agg(total_months = ("month", "count"), active_months = ("T", lambda x : (x > 0).sum())).reset_index()
district_monthly_counts.head()

,district,month,total_months,active_months
0,anantnag,1,1,1
1,anantnag,6,1,1
2,anantnag,9,1,1
3,anantnag,10,1,1
4,anantnag,11,1,1


In [171]:
district_monthly_counts["zero_months"] = district_monthly_counts["total_months"] - district_monthly_counts["active_months"]
district_monthly_counts["activity_ratio"] = district_monthly_counts["active_months"] / district_monthly_counts["total_months"]
district_monthly_counts["zero_month_ratio"] = district_monthly_counts["zero_months"] / district_monthly_counts["total_months"]

combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0
3,Jammu and Kashmir,anantnag,6,419,70,2,537.0,3257.0,3462.0,3897.0,491,3794.0,7359.0,11153.0,11644.0,1.0,0.0
4,Jammu and Kashmir,anantnag,9,861,104,1,455.0,2741.0,3951.0,2256.0,966,3196.0,6207.0,9403.0,10369.0,1.0,0.0


In [172]:
combined_df = combined_df.merge(district_monthly_counts[["district", "month", "activity_ratio", "zero_month_ratio"]], on = ["district", "month"], how = "left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio_x,zero_month_ratio_x,activity_ratio_y,zero_month_ratio_y
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0,1.0,0.0
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0,1.0,0.0
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0,1.0,0.0
3,Jammu and Kashmir,anantnag,6,419,70,2,537.0,3257.0,3462.0,3897.0,491,3794.0,7359.0,11153.0,11644.0,1.0,0.0,1.0,0.0
4,Jammu and Kashmir,anantnag,9,861,104,1,455.0,2741.0,3951.0,2256.0,966,3196.0,6207.0,9403.0,10369.0,1.0,0.0,1.0,0.0


In [173]:
district_volume_metrics = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrolment = ("E", "mean"),
                                               monthly_valatility = ("T", lambda x: x.std(ddof=0) / x.mean() if x.mean() > 0 else 0),
                                               peak_load_ratio = ("T", lambda x: x.max() / x.mean() if x.mean() > 0 else 0)).reset_index()


combined_df = combined_df.merge(district_volume_metrics, on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio_x,zero_month_ratio_x,activity_ratio_y,zero_month_ratio_y,avg_monthly_enrolment,monthly_valatility,peak_load_ratio
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0,1.0,0.0,538.0,0.000000,1.000000
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0,1.0,0.0,412.0,0.000000,1.000000
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0,1.0,0.0,668.5,0.544697,2.079181
3,Jammu and Kashmir,anantnag,6,419,70,2,537.0,3257.0,3462.0,3897.0,491,3794.0,7359.0,11153.0,11644.0,1.0,0.0,1.0,0.0,668.5,0.544697,2.079181
4,Jammu and Kashmir,anantnag,9,861,104,1,455.0,2741.0,3951.0,2256.0,966,3196.0,6207.0,9403.0,10369.0,1.0,0.0,1.0,0.0,668.5,0.544697,2.079181


In [174]:
district_update_burden = combined_df.groupby(["state", "district"]).agg(avg_monthly_enrollments = ("E", "sum"), avg_monthly_demo_updates = ("DU", "sum"), avg_monthly_bio_updates = ("BU", "sum")).reset_index()

district_update_burden["U"] = district_update_burden["avg_monthly_demo_updates"] + district_update_burden["avg_monthly_bio_updates"]
district_update_burden["biometric_burden"] = district_update_burden["avg_monthly_bio_updates"] / (district_update_burden["avg_monthly_bio_updates"] + district_update_burden["avg_monthly_demo_updates"])
district_update_burden["update_dominant"] = np.where(district_update_burden["U"] > district_update_burden["avg_monthly_enrollments"], 1, 0)
district_update_burden["enrollment_update_balance"] = district_update_burden["avg_monthly_enrollments"] / (district_update_burden["avg_monthly_enrollments"] + district_update_burden["U"])

combined_df = combined_df.merge(district_update_burden[["state", "district", "biometric_burden", "update_dominant", "enrollment_update_balance"]], on=["state", "district"], how="left")
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio_x,zero_month_ratio_x,activity_ratio_y,zero_month_ratio_y,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0,1.0,0.0,538.0,0.000000,1.000000,NaN,0,1.000000
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0,1.0,0.0,412.0,0.000000,1.000000,NaN,0,1.000000
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0,1.0,0.0,668.5,0.544697,2.079181,0.661141,1,0.053252
3,Jammu and Kashmir,anantnag,6,419,70,2,537.0,3257.0,3462.0,3897.0,491,3794.0,7359.0,11153.0,11644.0,1.0,0.0,1.0,0.0,668.5,0.544697,2.079181,0.661141,1,0.053252
4,Jammu and Kashmir,anantnag,9,861,104,1,455.0,2741.0,3951.0,2256.0,966,3196.0,6207.0,9403.0,10369.0,1.0,0.0,1.0,0.0,668.5,0.544697,2.079181,0.661141,1,0.053252


In [175]:
combined_df.drop(["activity_ratio_y", "zero_month_ratio_y"], axis=1, inplace=True)
combined_df.rename(columns={"activity_ratio_x": "activity_ratio", "zero_month_ratio_x": "zero_month_ratio"}, inplace=True)

In [176]:
combined_df = combined_df.groupby(["state", "district"], as_index = False).first()
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0,538.0,0.000000,1.000000,NaN,0,1.000000
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0,412.0,0.000000,1.000000,NaN,0,1.000000
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0,668.5,0.544697,2.079181,0.661141,1,0.053252
3,Jammu and Kashmir,bandipora,10,1,0,0,0.0,1.0,2.0,3.0,1,1.0,5.0,6.0,7.0,1.0,0.0,1.0,0.555556,1.555556,0.714286,1,0.222222
4,Jammu and Kashmir,bandipore,1,67,38,1,42.0,210.0,343.0,191.0,106,252.0,534.0,786.0,892.0,1.0,0.0,306.0,0.551783,1.993816,0.684275,1,0.067587


In [177]:
def normalize(x):
    maxx, minx = x.max(), x.min()
    if maxx == minx:
        return x * 0 + 0.5
    normalized = (x - minx) / (maxx - minx)
    return normalized

def inverse_normalize(x):
    inversed = 1 - normalize(x)
    return inversed

In [178]:
combined_df["access"] = (combined_df["activity_ratio"] + normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["responsiveness"] = normalize(combined_df["U"] / (combined_df["E"] + combined_df["U"]))
combined_df["inclusion"] = normalize((combined_df["age_0_5"] + combined_df["age_5_17"]) / combined_df["E"])
combined_df["stability"] = (inverse_normalize(combined_df["monthly_valatility"]) + inverse_normalize(combined_df["peak_load_ratio"])) / 2
combined_df["visibility"] = combined_df["activity_ratio"]

combined_df["DEI"] = (combined_df["access"] + combined_df["responsiveness"] + combined_df["inclusion"] + combined_df["stability"] + combined_df["visibility"]) / 5
combined_df["ASS"] = (inverse_normalize(combined_df["activity_ratio"]) + inverse_normalize(combined_df["avg_monthly_enrolment"])) / 2
combined_df["UBS"] = (normalize(combined_df["biometric_burden"]) + normalize(combined_df["update_dominant"])) / 2
combined_df["SRS"] = (normalize(combined_df["monthly_valatility"]) + normalize(combined_df["zero_month_ratio"])) / 2

combined_df.head()


,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,access,responsiveness,inclusion,stability,visibility,DEI,ASS,UBS,SRS
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0,538.0,0.000000,1.000000,NaN,0,1.000000,0.843037,0.000000,0.349071,1.000000,1.0,0.638422,0.406963,NaN,0.250000
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0,412.0,0.000000,1.000000,NaN,0,1.000000,0.762548,0.000000,0.000000,1.000000,1.0,0.552510,0.487452,NaN,0.250000
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0,668.5,0.544697,2.079181,0.661141,1,0.053252,0.926401,0.995609,1.000000,0.119137,1.0,0.808229,0.323599,0.628705,0.653244
3,Jammu and Kashmir,bandipora,10,1,0,0,0.0,1.0,2.0,3.0,1,1.0,5.0,6.0,7.0,1.0,0.0,1.0,0.555556,1.555556,0.714286,1,0.222222,0.500000,0.918744,1.000000,0.342842,1.0,0.752317,0.750000,0.762155,0.661282
4,Jammu and Kashmir,bandipore,1,67,38,1,42.0,210.0,343.0,191.0,106,252.0,534.0,786.0,892.0,1.0,0.0,306.0,0.551783,1.993816,0.684275,1,0.067587,0.694835,0.944494,0.805660,0.151672,1.0,0.719332,0.555165,0.686795,0.658489


In [179]:
combined_df.drop(["access", "responsiveness", "inclusion", "stability", "visibility"], axis=1, inplace=True)
combined_df.head()

,state,district,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,E,DU,BU,U,T,activity_ratio,zero_month_ratio,avg_monthly_enrolment,monthly_valatility,peak_load_ratio,biometric_burden,update_dominant,enrollment_update_balance,DEI,ASS,UBS,SRS
0,Jammu And Kashmir,doda,7,261,260,17,0.0,0.0,0.0,0.0,538,0.0,0.0,0.0,538.0,1.0,0.0,538.0,0.000000,1.000000,NaN,0,1.000000,0.638422,0.406963,NaN,0.250000
1,Jammu And Kashmir,shopian,7,308,84,20,0.0,0.0,0.0,0.0,412,0.0,0.0,0.0,412.0,1.0,0.0,412.0,0.000000,1.000000,NaN,0,1.000000,0.552510,0.487452,NaN,0.250000
2,Jammu and Kashmir,anantnag,1,180,36,0,83.0,549.0,1553.0,635.0,216,632.0,2188.0,2820.0,3036.0,1.0,0.0,668.5,0.544697,2.079181,0.661141,1,0.053252,0.808229,0.323599,0.628705,0.653244
3,Jammu and Kashmir,bandipora,10,1,0,0,0.0,1.0,2.0,3.0,1,1.0,5.0,6.0,7.0,1.0,0.0,1.0,0.555556,1.555556,0.714286,1,0.222222,0.752317,0.750000,0.762155,0.661282
4,Jammu and Kashmir,bandipore,1,67,38,1,42.0,210.0,343.0,191.0,106,252.0,534.0,786.0,892.0,1.0,0.0,306.0,0.551783,1.993816,0.684275,1,0.067587,0.719332,0.555165,0.686795,0.658489


In [180]:
combined_df.to_csv(r"D:\UIDAI hackathon\JandK\jandk_district_analysis.csv", index=False)
final_df = combined_df[["state", "district", "DEI", "ASS", "UBS", "SRS"]]
final_df.to_csv(r"D:\UIDAI hackathon\JandK\jandk_district_final_scores.csv", index=False)